# 删除多余文件

## 1. 导入工具包

In [ ]:
# 导入操作系统库以获取平台信息以及对目录操作的权限
import os

# 导入OpenCV2库以支持图像处理操作
import cv2

# 导入计算扩展库以支持数组运算
import numpy

# 导入进度条库的模块以令进程可视化
from tqdm import tqdm

# 导入图像处理库的图像模块以支持图像处理操作
from PIL import Image

## 2. 删除系统自动生成的多余文件

- ### Linux系统

In [ ]:
'''
查找格式：find /（查找范围） -name '查找关键字' -type d
find后的 . 代表当前目录
使用 -inname 能够忽略字符大小
'''

# Find命令递归查找当前目录是否有用于Mac系统用户预览的缓存文件夹（文件类似于Windows系统的Thumbs.db）
!find . -iname '__MACOSX'
#删除
!for i in `find . -iname '__MACOSX'`; do rm -rf $i; done
# 检查是否已删除
!find . -iname '__MACOSX'


# Find命令递归查找当前目录是否有用于Mac系统桌面服务存储的文件
!find . -iname '.DS_Store'
#删除
!for i in `find . -iname '.DS_Store'`; do rm -rf $i; done
# 检查是否已删除
!find . -iname '.DS_Store'


# Find命令递归查找当前目录是否有用于jupyter进行定点恢复的文件
!find . -iname '.ipynb_checkpoints'
#删除
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i; done
# 检查是否已删除
!find . -iname '.ipynb_checkpoints'

- ### Windows系统

In [ ]:
'''
查找格式：Get-ChildItem -Path '查找范围' "查找关键字"
-Path后的 . 代表当前目录
使用-Recurse参数以获取指定位置中的项目以及该位置的所有子项目。
使用-Filter参数以指定用于限定-Path参数的过滤器。
管道符 | 会把前一个命令的对象传递给第二个命令所对应支持ByValue的参数，然后执行
'''

# Get-ChildItem命令递归查找当前目录是否有用于Mac系统用户预览的缓存文件夹（文件类似于Windows系统的Thumbs.db）并递归强制删除
os.system('Get-ChildItem -Path . -Recurse -Filter "__MACOSX" | Remove-Item -Force')
# 检查是否已删除
os.system('Test-Path -Path . -Filter "__MACOSX"')


# Get-ChildItem命令递归查找当前目录是否有用于Mac系统桌面服务存储的文件
os.system('Get-ChildItem -Path . -Recurse -Filter ".DS_Store" | Remove-Item -Force')
# 检查是否已删除
os.system('Test-Path -Path . -Filter ".DS_Store"')


# Get-ChildItem命令递归查找当前目录是否有用于jupyter进行定点恢复的文件
os.system('Get-ChildItem -Path . -Recurse -Filter ".ipynb_checkpoints" | Remove-Item -Force')
# 检查是否已删除
os.system('Test-Path -Path . -Filter ".ipynb_checkpoints"')

- ### 其它

In [ ]:
# Python命令查找指定目录下是否有带目标后缀的文件
'%目标后缀%' in os.listdir('%指定目录%')

## 3. 删除无法读取的文件与非三通道的文件

- ### Unix / Windows 系统

In [ ]:
'''
cv2.imread(path, flag) ：opencv中的处理图片的函数，需注意path不能有中文，否则会返回None
path : 以字节为单位接收到的图像数据
flag : 指定读取图像的方式，具体见下：
cv2.IMREAD_COLOR (实参1) 表示以忽略alpha通道的彩色模式加载图片（默认参数）
cv2.IMREAD_GRAYSCALE (实参0) 表示以灰度模式加载图片
cv2.IMREAD_UNCHANGED (实参-1) 表示以包括alpha通道的完整模式加载图片

为解决中文路径的问题，引入以下函数以替代imread函数：

cv2.imdecode(buf, flag) : 用于从内存缓存中读取图像数据并将其转换为图像格式
buf : 以字节为单位接收到的图像数据
flag : 指定读取图像的方式（同上）

numpy.fromfile(file, dtype = float, count = -1, sep = '', offset = 0, *, like = None) : 从文本或二进制文件中的数据构造一个数组
file ：需读取的文件对象的路径
dtype ：返回数组的数据类型
count : 要读取的数据长度（默认为-1，表示所有数据）
sep : 数据之间的分隔符（默认为''空分隔符，表示文件应被视为二进制文件）
offset : 与文件当前位置的以字节为单位的偏移量（默认为0且仅允许用于二进制文件）
like : 创建非NumPy数组的数组引用对象（默认为None，表示不允许创建）

numpy.uint8 : 将数组的类型转换为uint8，即表示范围[0, 255]整数的无符号八位整型

由此可得解决方案：cv2.imdecode(numpy.fromfile(r"/path/to/image", dtype = numpy.uint8), flag = -1) # 路径前加上r能够防止\的转义，亦或者使用\\来实现


Image.open(path) ：PIL中的处理图片的函数，注意使用时需 from PIL import Image （或者 Import PIL 之后 img = PIL.Image.open(path)）
同时需注意：
当图像格式为RGBA时，Image.open()读取的格式为RGBA（其中A表示图像的alpha通道，即RGBA共四个通道）；而cv2.imread()读取的格式是BGR，只有三个通道。故使用PIL进行处理非三通道检测处理。
'''

# 指定工作目录
WorkDir = 'DataSet' #WorkDir = os.getcwd() # 指定根目录为工作目录

for Subfolder in tqdm(os.listdir(WorkDir)): # 遍历工作目录下包含的子文件夹并让进度可视化
    for File in os.listdir(os.path.join(WorkDir, Subfolder)): # 合并工作目录、子文件夹得到子文件夹路径以遍历路径下包含的文件
        File_Path = os.path.join(WorkDir, Subfolder, File) # 合并工作目录、子文件夹、文件名得到文件路径

        # 先判断是否可以被opencv读取
        IsImageReadable = cv2.imdecode(numpy.fromfile(File_Path, dtype = numpy.uint8), flag = 1) # 以彩色模式读入路径对应的图片
        if IsImageReadable is None: # 可读取的文件通常返回为numpy.array类型，其余返回None（包括opencv不支持的gif格式文件）
            print(File_Path, '判定为不明文件，执行删除')
            os.remove(File_Path)
        else:
            # 再对可读取文件判断是否为非三通道文件
            IsImageRGB = numpy.array(Image.open(File_Path)) # 由于要使用img.shape函数，故打开图片后需将image形式转换成array数组
            try: # 尝试获取图像通道信息并判定通道数是否为三
                Channel = IsImageRGB.shape[2] # [0]返回水平像素数；[1]返回垂直像素数；[2]返回通道数
                if Channel != 3:
                    print(File_Path, '判定为非三通道文件，执行删除')
                    os.remove(File_Path)
            except:
                print(File_Path, '判定为不明文件，执行删除')
                os.remove(File_Path)

## 4. 再次删除多余的`.ipynb_checkpoints`目录

- ### Linux系统

In [ ]:
# Find命令递归查找当前目录是否有用于jupyter进行定点恢复的文件
!find . -iname '.ipynb_checkpoints'
#删除
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i; done
# 验证是否已删除
!find . -iname '.ipynb_checkpoints'

- ### Windows系统

In [ ]:
# Get-ChildItem命令递归查找当前目录是否有用于jupyter进行定点恢复的文件
os.system('Get-ChildItem -Path . -Recurse -Filter ".ipynb_checkpoints" | Remove-Item -Force')
# 检查是否已删除
os.system('Test-Path -Path . -Filter ".ipynb_checkpoints"')